
    ## 导学单元

    ### 课程目标
    - 快速回顾Budyko曲线的基本思想，理解完整工作流的输入输出。
    - 熟悉Notebook与脚本 `examples/complete_workflow_example.py` 的配合方式，便于在命令行与交互环境间切换。

    ### 前置知识
    - 基础 Python（NumPy/Pandas）与 Matplotlib/Seaborn 绘图经验。
    - 对能量-水量平衡、蒸散发与降水等基本水文气候概念的认知。

    ### 所需包和运行环境
    - Python 3.9+，建议在虚拟环境或 Conda 环境中运行 Jupyter Notebook。
    - 依赖包主要列于项目 `requirements.txt`，常用库包括 NumPy、Pandas、Matplotlib、Seaborn、Xarray、SciPy（按需）。
    - 如需加速或批量运行，可直接在命令行调用 `examples/complete_workflow_example.py` 以生成对应输出。

    ### 学习路径与章节骨架
    1. 背景/概念：快速回顾Budyko曲线的基本思想，理解完整工作流的输入输出。
    2. 数据加载与预览：理解数据来源、字段含义与基本统计特征。
    3. 核心方法步骤（配合 `examples/complete_workflow_example.py`）：按照脚本顺序完成数据准备、参数估计、曲线拟合与误差评估，形成闭环。
    4. 结果可视化：输出关键图表（曲线拟合、误差分布、敏感性分析），支撑结论陈述。
    5. 反思与扩展练习：结合自身数据需求，思考如何定制参数化形式或替换拟合算法。

    ### 你将学到什么
    - 从零搭建可复现的Budyko分析工作流的步骤与依赖。
- 在Notebook中调试、记录并验证脚本函数的调用方式。
- 如何根据误差和可视化反馈调整模型假设。

    ### 本节完成标志
    - [ ] 顺利跑通脚本对应的全流程并保存生成的图表或中间结果。
- [ ] 对每个核心函数的输入输出有清晰认识，可复述其作用。


# complete_workflow_example.py Notebook

完整的Budyko分析工作流示例

本示例演示了从数据加载到结果输出的完整流程，
重点展示径流数据(Q)在整个分析中的核心作用。

工作流程：
1. 数据准备（P, Q, 气象数据, LAI, CO2）
2. PET计算（传统方法 vs LAI+CO2创新方法）
3. 水量平衡计算（基于径流Q）
4. Budyko曲线拟合
5. 偏差计算和分析
6. 轨迹分析
7. 驱动因子归因
8. 可视化和报告

注意：本示例使用模拟数据。使用真实数据时，请替换数据加载部分。

In [2]:
# examples/complete_workflow_example.py

from pathlib import Path
import sys

# 在 Notebook 中，使用 Path.cwd() 获取当前工作目录
# 假设 notebook 在 examples/ 目录下，src/ 在父目录
project_root = Path.cwd()
if project_root.name == 'examples':
    # 如果当前在 examples 目录，向上一级找到项目根目录
    project_root = project_root.parent

src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 导入核心模块
from budyko.water_balance import WaterBalanceCalculator, RunoffAnalyzer
from budyko.curves import BudykoCurves
from budyko.deviation import DeviationAnalysis, TemporalStability
from budyko.trajectory_jaramillo import TrajectoryAnalyzer, MovementStatistics
from models.pet_lai_co2 import PETWithLAICO2, PETComparator
from models.pet_models import PETModelFactory
from analysis.deviation_attribution import DeviationAttributor

# 设置
sns.set_style('whitegrid')

# 导入中文字体配置
try:
    from utils.plotting_config import setup_chinese_fonts
    setup_chinese_fonts()
except ImportError:
    # 备用配置
    plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['figure.dpi'] = 100

# 创建输出目录
output_dir = project_root / 'outputs' / 'complete_workflow'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"✓ 环境准备完成")
print(f"项目根目录: {project_root}")
print(f"输出目录: {output_dir}")

✓ 中文字体配置已应用
✓ 环境准备完成
项目根目录: c:\Users\ASUS\OneDrive\SCI\Github\Budyko-Analysis
输出目录: c:\Users\ASUS\OneDrive\SCI\Github\Budyko-Analysis\outputs\complete_workflow
